In [41]:
from contextlib import contextmanager
import sqlite3
import yaml

In [42]:
curr_dir = ! pwd
curr_dir = curr_dir[0]
print(curr_dir)
assert curr_dir.endswith("hyperborea3/scripts/spells")

/home/jeremy/projects/hyperborea3/scripts/spells


In [43]:
load_folder = "lvl4"

In [44]:
@contextmanager
def db_cur():
    URI = f"../../hyperborea3/hyperborea.sqlite3"
    con = sqlite3.connect(URI, check_same_thread=False)
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    yield cur
    con.commit()
    con.close()

Test connection to db

In [45]:
with db_cur() as cur:
    cur.execute("""
        SELECT *
          FROM spells;
    """)
    result = [dict(x) for x in cur.fetchall()]
result[:1]

[{'spell_id': 1,
  'spell_name': 'Acid Arrow',
  'rng': '30 feet',
  'dur': 'special',
  'reversible': 0,
  'pp': 175,
  'spell_desc': '<p>A magic arrow darts from the finger of the caster. On a successful attack roll (<i>dexterity</i> modifier applies), the <i>acid arrow</i> strikes for 1d4+1 hp physical damage, plus an additional 2d4 hp acid damage in the same round. Magicians (but not other sorcerers) enjoy a +1 bonus to the attack roll for every 2 CA levels (CA 3–4 = +2, CA 5–6 = +3, and so forth). Acid damage will persist for higher level sorcerers:</p> <ul><li>1 extra round for CA 4–6</li> <li>2 extra rounds for CA 7–9</li> <li>3 extra rounds for CA 10 or greater.</li></ul> <p>For example, an <i>acid arrow</i> fired by a CA 12 sorcerer on round 1 would inflict 1d4+1 hp base damage plus 2d4 hp acid damage on round 1, 2d4 hp acid damage on round 2, 2d4 hp acid damage on round 3, and a final 2d4 hp acid damage on round 4. The acid may ruin armour or clothing per referee discretion. 

In [46]:
files = ! ls {load_folder}/*.yml
files

['lvl4/Black_Tentacles.yml',
 'lvl4/Boil.yml',
 'lvl4/Brain_Death.yml',
 'lvl4/Brink_of_Death.yml',
 'lvl4/Change_Temperature.yml',
 'lvl4/Charm_Monster.yml',
 'lvl4/Cone_of_Cold.yml',
 'lvl4/Confusion.yml',
 'lvl4/Contagion.yml',
 'lvl4/Control_Water.yml',
 'lvl4/Controlled_Blast_Fireball.yml',
 'lvl4/Cure_Serious_Burns.yml',
 'lvl4/Cure_Serious_Wounds.yml',
 'lvl4/Dig_Hole.yml',
 'lvl4/Dimension_Door.yml',
 'lvl4/Discern_Lie.yml',
 'lvl4/Divination.yml',
 'lvl4/Dweomered_Weapon.yml',
 'lvl4/Ecstasy_of_Shadow.yml',
 'lvl4/Emotion.yml',
 'lvl4/Enervation.yml',
 'lvl4/Extend_Spell_I.yml',
 'lvl4/Faithful_Hound.yml',
 'lvl4/False_Vacancy.yml',
 'lvl4/Fire_Shield.yml',
 'lvl4/Globe_of_Invulnerability,_Lesser.yml',
 'lvl4/Hold_Vegetation.yml',
 'lvl4/Ice_Javelin.yml',
 'lvl4/Ice_Storm.yml',
 'lvl4/Igloo.yml',
 'lvl4/Illusory_Wall.yml',
 'lvl4/Imbue_with_Spell_Ability.yml',
 'lvl4/Improved_Invisibility.yml',
 'lvl4/Inflict_Madness.yml',
 'lvl4/Insect_Plague.yml',
 'lvl4/Lifeglobe.yml',
 'lv

In [47]:
def load_yml_spell(file_name):
    with open(file_name, "r") as f:
        spell = yaml.safe_load(f)
    return spell

In [48]:
def spell_preload_check(spell):
    with db_cur() as cur:
        cur.execute(
            """
            SELECT *
            FROM spells
            WHERE spell_id = ?;
            """,
            (spell["id"],),
        )
        db_spell = cur.fetchone()
    assert spell["name"] == db_spell["spell_name"]
    assert isinstance(spell["id"], int)
    assert isinstance(spell["pp"], int)
    assert isinstance(spell["Rng"], (str, int))
    assert isinstance(spell["Dur"], str)
    assert spell["Rev"] in [0, 1]
    assert isinstance(spell["Desc"], str)
    assert spell["Desc"][:3] == "<p>"
    if spell["id"] not in [53]:
        assert spell["Desc"][-4:] == "</p>"



In [49]:
def update_spell(spell):
    with db_cur() as cur:
        cur.execute(
            """
            UPDATE spells
            SET rng = ?
                , dur = ?
                , reversible = ?
                , pp = ?
                , spell_desc = ?
            WHERE spell_id = ?;
            """,
            (
                spell["Rng"],
                spell["Dur"],
                spell["Rev"],
                spell["pp"],
                spell["Desc"],
                spell["id"],
            ),
        )
    return 

### Test parsing all the files

In [52]:
for f in files:
    print(f"checking {f}")
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    print("success")
    print()

checking lvl4/Black_Tentacles.yml
success

checking lvl4/Boil.yml
success

checking lvl4/Brain_Death.yml
success

checking lvl4/Brink_of_Death.yml
success

checking lvl4/Change_Temperature.yml
success

checking lvl4/Charm_Monster.yml
success

checking lvl4/Cone_of_Cold.yml
success

checking lvl4/Confusion.yml
success

checking lvl4/Contagion.yml
success

checking lvl4/Control_Water.yml
success

checking lvl4/Controlled_Blast_Fireball.yml
success

checking lvl4/Cure_Serious_Burns.yml
success

checking lvl4/Cure_Serious_Wounds.yml
success

checking lvl4/Dig_Hole.yml
success

checking lvl4/Dimension_Door.yml
success

checking lvl4/Discern_Lie.yml
success

checking lvl4/Divination.yml
success

checking lvl4/Dweomered_Weapon.yml
success

checking lvl4/Ecstasy_of_Shadow.yml
success

checking lvl4/Emotion.yml
success

checking lvl4/Enervation.yml
success

checking lvl4/Extend_Spell_I.yml
success

checking lvl4/Faithful_Hound.yml
success

checking lvl4/False_Vacancy.yml
success

checking lvl4/

Do it for real

In [53]:
for f in files:
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    update_spell(spell)

In [54]:
load_yml_spell(files[-1])

{'id': 441,
 'name': 'Ward Off Insects',
 'pp': 245,
 'Lvl': ['drd 4'],
 'Rng': 0,
 'Dur': '1 turn per CA level',
 'Rev': 0,
 'Desc': '<p>Evokes an invisible barrier to form around the caster in a 10-foot radius, repelling all normal insect life. Giant insects are allowed <i>sorcery</i> saving throws. Successful saves allow the creatures to behave as normal, though they suffer 1d6 hp damage for entering the field; failed saves cause the insects to flee in fear. This spell does not affect spiders and centipedes.</p>'}